In [2]:
import re
import spacy
from spacy.tokens import Span

In [4]:
text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."
pattern = r"Paul [A-Z]\w+"


In [12]:
nlp = spacy.blank("en")
doc = nlp(text)
original_ents = list(doc.ents)
print(doc.ents)
mwt_ents = []
for match in re.finditer(pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    print(span)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))

for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label="PERSON")
    original_ents.append(per_ent)

doc.ents = original_ents
print(doc.ents)

for ent in doc.ents:
    print(ent.text, ent.label_)

()
Paul Newman
Paul Hollywood
(Paul Newman, Paul Hollywood)
Paul Newman PERSON
Paul Hollywood PERSON


In [13]:
# here we have token addressing in here instead of character addressing
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]


In [15]:
from spacy.language import Language

@Language.component("paul_ner")
def paul_ner(doc):
    pattern = r"Paul [A-Z]\w+"
    original_ents = list(doc.ents)
    mwt_ents = []
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))

    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="PERSON")
        original_ents.append(per_ent)

    doc.ents = original_ents
    return (doc)


In [16]:
nlp2 = spacy.blank("en")
nlp2.add_pipe("paul_ner")


<function __main__.paul_ner(doc)>

In [17]:
doc2 = nlp2(text)
print(doc2.ents)

(Paul Newman, Paul Hollywood)


In [19]:
from spacy.language import Language
from spacy.util import filter_spans

@Language.component("cinema_ner")
def cinema_ner(doc):
    pattern = r"Hollywood"
    original_ents = list(doc.ents)
    mwt_ents = []
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))

    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="CINEMA")
        original_ents.append(per_ent)

    filtered = filter_spans(original_ents)
    doc.ents = filtered
    return (doc)


In [20]:
nlp3 = spacy.load("en_core_web_sm")
nlp3.add_pipe("cinema_ner")


<function __main__.cinema_ner(doc)>

In [22]:
doc3 = nlp3(text)
for ent in doc3.ents:
    print(ent.text, ent.label_)


Paul Newman PERSON
American NORP
Paul Hollywood PERSON
British NORP
